In [12]:
import os
import config as cfg
# read in the header

filename = 'frt00003bfb_07_if166j_mtr3.hdr'
filepath = os.path.join(cfg.DATA_DIR, filename)
with open(filepath) as file:
    lines = [line.strip() for line in file]

# hdr = {}
# for line in lines:
#     if '=' in line:
#         pair = line.split('=')
#         hdr.update({pair[0].strip(): pair[1].strip()})
# print(hdr)

{'description': '{', 'samples': '851', 'lines': '797', 'bands': '489', 'header offset': '0', 'file type': 'ENVI Standard', 'data type': '4', 'interleave': 'bsq', 'sensor type': 'Unknown', 'byte order': '0', 'map info': '{MRO Mars Equirectangular [IAU 2000] [20.00N; 0.00E], 1.0000, 1.0000, -1020964.3000, 1339036.0000, 1.8000000000e+01, 1.8000000000e+01, D_Mars_2000_IAU_IAG_custom_sphere_lat20, units', 'projection info': '{17, 3393833.3, 20.000000, 0.000000, 0.0, 0.0, D_Mars_2000_IAU_IAG_custom_sphere_lat20, MRO Mars Equirectangular [IAU 2000] [20.00N; 0.00E], units', 'coordinate system string': '{PROJCS["MRO Mars Equirectangular [IAU 2000] [20.00N; 0.00E]",GEOGCS["GCS_Mars_2000_IAU_IAG_custom_sphere_lat20",DATUM["D_Mars_2000_IAU_IAG_custom_sphere_lat20",SPHEROID["Mars_2000_IAU_IAG_custom_sphere_lat20",3393833.260758386,0.0]],PRIMEM["Reference_Meridian",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Equidistant_Cylindrical"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",

In [1]:
%matplotlib qt
import os
import config as cfg
import hyperspy.api as hs
from skimage import io
import sklearn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
'''
Import the image and set axes labels
'''

# read in tiff image
imgname = 'frt00009a16_data_cube.tif'
imgpath = os.path.join(cfg.DATA_DIR, imgname)
img = io.imread(imgpath)

# get shape and transpose channels to end dimension
c, w, h = img.shape
img = img.transpose(1, 2, 0)

# remove values above 1
img[img > 1] = 0

axes_x = {'name': 'x', 'size': img.shape[0], 'units': 'px'}
axes_y = {'name': 'y', 'size': img.shape[1], 'units': 'px'}
axes_w = {'name': 'wavelength band', 'size': img.shape[2], 'units': 'index'}

# convert image to signal object
im = hs.signals.Signal1D(img, axes=[axes_x, axes_y, axes_w])
im.axes_manager
im.plot()

/Users/maxhipperson/.virtualenvs/marsnet-ZEfWMKGE/lib/python3.6/site-packages/skimage/external/tifffile/tifffile.py:2128: UserWarning: tags are not ordered by code
  warnings.warn("tags are not ordered by code")


In [5]:
'''
PCA
'''

im.decomposition()
im.learning_results.summary()

Decomposition parameters:
-------------------------

Decomposition algorithm : 	svd
Poissonian noise normalization : False
Output dimension : None
Centre : None


In [7]:
im.plot_explained_variance_ratio(threshold=0.005)

In [8]:
im.plot_decomposition_results()

In [9]:
imc = im.get_decomposition_model(components=6)

# calculate and display residuals
(im - imc).plot()

In [9]:
# independent componant analysis
im.blind_source_separation(number_of_components=10)
im.learning_results.summary()

Decomposition parameters:
-------------------------

Decomposition algorithm : 	svd
Poissonian noise normalization : False
Output dimension : None
Centre : None

Demixing parameters:
------------------------
BSS algorithm : sklearn_fasticaNumber of components : 10


/Users/maxhipperson/.virtualenvs/marsnet-ZEfWMKGE/lib/python3.6/site-packages/scipy/linalg/basic.py:1321: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  x, resids, rank, s = lstsq(a, b, cond=cond, check_finite=False)


In [10]:
im.plot_bss_results()

In [11]:
# online PCA
im.decomposition(algorithm='ORPCA',
                 output_dimension=20,
                 method='MomentumSGD',
                 learning_rate=1.1,
                 momentum=0.5)
im.plot_explained_variance_ratio()

In [14]:
# different decomposition algorithms

algorithms = [
    'RPCA_GoDec',
    'ORPCA',
    'nmf'
]

for index, algorithm in enumerate(algorithms):
    if index in [0, 1]:
        im.decomposition(algorithm=algorithm, output_dimension=3)
    else:
        im.decomposition(algorithm=algorithm)
    im.plot_explained_variance_ratio()

In [13]:
im.plot_decomposition_results()

In [20]:
im.learning_results.summary()

Decomposition parameters:
-------------------------

Decomposition algorithm : 	ORPCA
Poissonian noise normalization : False
Output dimension : 20
Centre : None

Demixing parameters:
------------------------
BSS algorithm : sklearn_fasticaNumber of components : 10


In [21]:
im.blind_source_separation(10)

In [22]:
im.plot_bss_results()

In [24]:
imc = im.get_decomposition_model(components=6)

# calculate and display residuals
(im - imc).plot()

In [5]:
sc = sklearn.cluster.SpectralClustering()
sc.fit(im)
y_pred = sc.labels_
print(y_pred)

AttributeError: module 'sklearn' has no attribute 'cluster'

In [ ]:

sc.fit_predict()

AttributeError: module 'sklearn' has no attribute 'cluster'